In [95]:
from razdel import tokenize, sentenize
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)
import re
import csv

In [103]:
with open('NEWSRU.csv', newline='') as csvfile:
    csv_reader = csv.reader(csvfile, delimiter=';')
    articles = []
    sections = []
    for line in csv_reader:
        articles.append(line[3])
        sections.append(line[4])
print(articles[1:3])
print(sections[1:3])

['Глава программы ВОЗ по чрезвычайным ситуациям в области здравоохранения Майк Райан предупредил, что вакцины от коронавируса COVID-19 не позволят свести на ноль распространение инфекции, передает "Интерфакс". "Вакцины не означают полное отсутствие COVID-19", - сказал Райан.Глава ВОЗ Тедрос Гебрейесус  отметил, что "прогресс в области разработки вакцин помогает нам, и мы начинаем видеть свет в конце туннеля".  "Однако в ВОЗ обеспокоены тем, что создается все большее ощущение, что пандемия закончилась, но на самом деле она продолжается", - отметил гендиректор организации."Мы знаем, что это был тяжелый год. Люди устали. Больницы работают на пределе возможностей или даже сверх этого предела", - сказал Гебрейесус.За время пандемии в мире зафиксировано почти 66 млн случаев заражения коронавирусом, более 1,5 млн человек скончались. Около 45 млн выздоровели', 'Представители Евросоюза и Великобритании не договорились по поводу соглашения о свободной торговле после выхода Соединенного Королевст

In [97]:
sentences = list(sentenize(str(articles)))
sentences[:4]

[Substring(0,
           225,
           '[\'Текст статьи\', \'Глава программы ВОЗ по чрезвычайным ситуациям в области здравоохранения Майк Райан предупредил, что вакцины от коронавируса COVID-19 не позволят свести на ноль распространение инфекции, передает "Интерфакс".'),
 Substring(226,
           428,
           '"Вакцины не означают полное отсутствие COVID-19", - сказал Райан.Глава ВОЗ Тедрос Гебрейесус  отметил, что "прогресс в области разработки вакцин помогает нам, и мы начинаем видеть свет в конце туннеля".'),
 Substring(430,
           628,
           '"Однако в ВОЗ обеспокоены тем, что создается все большее ощущение, что пандемия закончилась, но на самом деле она продолжается", - отметил гендиректор организации."Мы знаем, что это был тяжелый год.'),
 Substring(629, 641, 'Люди устали.')]

In [98]:
emb = NewsEmbedding()
segmenter = Segmenter()

In [6]:
vars(doc_data.sents[2])

{'start': 430,
 'stop': 628,
 'text': '"Однако в ВОЗ обеспокоены тем, что создается все большее ощущение, что пандемия закончилась, но на самом деле она продолжается", - отметил гендиректор организации."Мы знаем, что это был тяжелый год.',
 'tokens': [DocToken(start=430, stop=431, text='"'),
  DocToken(start=431, stop=437, text='Однако'),
  DocToken(start=438, stop=439, text='в'),
  DocToken(start=440, stop=443, text='ВОЗ'),
  DocToken(start=444, stop=455, text='обеспокоены'),
  DocToken(start=456, stop=459, text='тем'),
  DocToken(start=459, stop=460, text=','),
  DocToken(start=461, stop=464, text='что'),
  DocToken(start=465, stop=474, text='создается'),
  DocToken(start=475, stop=478, text='все'),
  DocToken(start=479, stop=486, text='большее'),
  DocToken(start=487, stop=495, text='ощущение'),
  DocToken(start=495, stop=496, text=','),
  DocToken(start=497, stop=500, text='что'),
  DocToken(start=501, stop=509, text='пандемия'),
  DocToken(start=510, stop=521, text='закончилась'),

In [8]:
ner_tagger = NewsNERTagger(emb)
doc_data.tag_ner(ner_tagger)
display(doc_data.spans[:5])

[DocSpan(start=34, stop=37, type='ORG', text='ВОЗ', tokens=[...]),
 DocSpan(start=90, stop=100, type='PER', text='Майк Райан', tokens=[...]),
 DocSpan(start=214, stop=223, type='ORG', text='Интерфакс', tokens=[...]),
 DocSpan(start=285, stop=290, type='PER', text='Райан', tokens=[...]),
 DocSpan(start=297, stop=300, type='ORG', text='ВОЗ', tokens=[...])]

In [9]:
doc_data.ner.spans[:3]

[Span(
     start=34,
     stop=37,
     type='ORG'
 ),
 Span(
     start=90,
     stop=100,
     type='PER'
 ),
 Span(
     start=214,
     stop=223,
     type='ORG'
 )]

In [102]:
from collections import Counter
from operator import itemgetter

data = list(zip(articles[1:], sections[1:]))
data = sorted(data,  key=itemgetter(1))
labels_distrib = Counter()
ner_distrib = Counter()

for i in range(1, len(data)):
    cur = data[i]
    text = cur[0]
    sect = cur[1]
    #emb = NewsEmbedding()
    #segmenter = Segmenter()
    doc_article = Doc(text)
    doc_article.segment(segmenter)
    ner_tagger = NewsNERTagger(emb)
    if sect != data[i-1][1]:
        print("Предыдущая рубрика: ", data[i-1][1], "Текущая рубрика: ", sect)
        print("Распределение именованных сущностей в рубрике {0}: {1}".format(data[i-1][1], dict(labels_distrib)))
        print("Распределение представителей именованных сущностей в рубрике {0}: {1}".format(data[i-1][1], dict(list(ner_distrib.most_common())[:30])))
        ner_distrib.clear() 
        labels_distrib.clear()
        
    doc_article.tag_ner(ner_tagger)    
    ner_distrib.update(span.text for span in doc_article.spans)
    labels_distrib.update(span.type for span in doc_article.spans)

Предыдущая рубрика:  Автоновости Текущая рубрика:  В России
Распределение именованных сущностей в рубрике Автоновости: {'LOC': 4869, 'ORG': 8388, 'PER': 2121}
Распределение представителей именованных сущностей в рубрике Автоновости: {'России': 877, 'ГИБДД': 316, 'Москве': 277, 'Tesla': 267, 'Nissan': 214, 'Москвы': 195, 'Volkswagen': 177, 'МВД': 176, 'Renault': 170, 'РБК': 169, 'АвтоВАЗ': 141, 'Hyundai': 140, 'Коммерсант': 135, 'США': 135, 'Motor.ru': 128, 'РФ': 123, 'Kia': 119, 'BMW': 116, 'ТАСС': 111, 'Toyota': 102, 'АвтоВАЗа': 95, 'Гон': 89, 'Росстандарта': 86, 'АЕБ': 80, 'Автостат': 77, 'Uber': 77, 'Ассоциации европейского бизнеса (АЕБ)': 73, 'Китае': 67, 'Volvo': 65, 'Ford': 64}
Предыдущая рубрика:  В России Текущая рубрика:  В мире
Распределение именованных сущностей в рубрике В России: {'LOC': 72746, 'ORG': 59300, 'PER': 56891}
Распределение представителей именованных сущностей в рубрике В России: {'России': 6252, 'РФ': 5775, 'Москве': 2739, 'Москвы': 2373, 'СМИ': 1394, 'ФСБ': 1